In [1]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import copy
import shap
import pickle
import os

from scipy import linalg
from scipy.special import expit
from scipy import stats
from tqdm import tqdm
from matplotlib import cm
from sklearn.base import TransformerMixin, ClassifierMixin
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.metrics import (roc_auc_score, f1_score, precision_score, recall_score, 
                             RocCurveDisplay, PrecisionRecallDisplay)
from pandas.api.types import CategoricalDtype

In [2]:
# Import MICE functions
from mice_functions import *

## Load incomplete dataset

In [3]:
# Load incomplete dataset
props_name = [10, 20, 30, 40, 50]
boston_path = "../../data/toy-dataset/boston-processed/"
biopsy_path = "../../data/toy-dataset/biopsy-processed/"

# Starting with boston dataset
boston_incomp = {}
for p in props_name:
    boston_incomp[p] = pd.read_csv(boston_path + "boston_{}.csv".format(p))


# Followed by biopsy dataset
biopsy_incomp = {}
for p in props_name:
    biopsy_incomp[p] = pd.read_csv(biopsy_path + "biopsy_{}.csv".format(p))

In [4]:
# List down target variables for imputation
boston_targets_cat = ["chas"]
boston_targets_num = ["nox"]

biopsy_targets_cat = []
biopsy_targets_num = ["V1", "V2", "V3"]

## Apply MICE algorithm
### General setup

In [5]:
# Select imputation models
method_cat = "pmm"
method_num = "pmm"

# Set number of imputations and maximum number of iteration
maxit = 40
m = 20

# Set number of donors for PMM
d = 10

# Set random seed
SEED = 2023

# Path to store any plots assessing convergence
conv_path = "../../results/figures/python_convergence/"
if not os.path.exists(conv_path):
    os.mkdir(conv_path)

### `boston` dataset

In [6]:
# Prevent plots from being printed
%matplotlib agg

# Prepare dictionary to store imputed data
boston_imputed = {}

for p in props_name:
    print("Applying on dataset {}".format(p))
    
    # Apply MICE algorithm
    boston_imputed[p] = MICE(boston_incomp[p], boston_targets_cat, boston_targets_num, 
                             m=m, maxit=maxit, d=d, seed=SEED, 
                             method_cat=method_cat, method_num=method_num)
    
    # Construct trace plot
    f = ChainStatsViz(boston_imputed[p])
    f.savefig(conv_path + "boston_traceplot_{}.pdf".format(p))
    
    # Construct strip plots for imputed variables
    for c in ["chas", "nox"]:
        f = plotImputedData(boston_imputed[p], c)
        f.savefig(conv_path + "boston_{}_stripplot_{}.pdf".format(c, p))
    
plt.clf()

Applying on dataset 10


100%|███████████████████████████████████████████| 40/40 [00:06<00:00,  6.32it/s]


Applying on dataset 20


100%|███████████████████████████████████████████| 40/40 [00:07<00:00,  5.09it/s]


Applying on dataset 30


100%|███████████████████████████████████████████| 40/40 [00:08<00:00,  4.77it/s]


Applying on dataset 40


100%|███████████████████████████████████████████| 40/40 [00:08<00:00,  4.72it/s]


Applying on dataset 50


100%|███████████████████████████████████████████| 40/40 [00:09<00:00,  4.41it/s]


In [7]:
# Save multiply imputed data as Python objects
with open(boston_path + "imputed.pickle", "wb") as handle:
    pickle.dump(boston_imputed, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [8]:
# Retrieve chain statistics from last iteration for the report
for p in props_name:
    print("p = {}".format(p / 100))
    tempmu = np.median(boston_imputed[p]["chainmean"][3:5, :, -1], axis=1)
    print("Posterior median for mu = {}".format(tempmu))
    tempsig = np.median(boston_imputed[p]["chainstd"][3:5, :, -1], axis=1)
    print("Posterior median for sigma = {}".format(tempsig))

p = 0.1
Posterior median for mu = [0.02777778 0.62842857]
Posterior median for sigma = [0.11785113 0.07429757]
p = 0.2
Posterior median for mu = [0.3902439  0.62602778]
Posterior median for sigma = [0.49321571 0.05346983]
p = 0.3
Posterior median for mu = [0.28461538 0.60459145]
Posterior median for sigma = [0.43011613 0.06085357]
p = 0.4
Posterior median for mu = [0.42708333 0.62543764]
Posterior median for sigma = [0.49548083 0.07857837]
p = 0.5
Posterior median for mu = [0.08333333 0.54083024]
Posterior median for sigma = [0.27713021 0.05057828]


### `biopsy` dataset

In [9]:
# Prepare dictionary to store imputed data
biopsy_imputed = {}

for p in props_name:
    print("Applying on dataset {}".format(p))
    
    # Apply MICE algorithm
    biopsy_imputed[p] = MICE(biopsy_incomp[p], biopsy_targets_cat, biopsy_targets_num, 
                             m=m, maxit=maxit, d=d, seed=SEED, 
                             method_cat=method_cat, method_num=method_num)
    
    # Construct trace plot
    f = ChainStatsViz(biopsy_imputed[p])
    f.savefig(conv_path + "biopsy_traceplot_{}.pdf".format(p))
    
    # Construct strip plots for imputed variables
    for c in ["V1", "V2", "V3"]:
        f = plotImputedData(biopsy_imputed[p], c)
        f.savefig(conv_path + "biopsy_{}_stripplot_{}.pdf".format(c, p))

plt.clf()

Applying on dataset 10


100%|███████████████████████████████████████████| 40/40 [00:09<00:00,  4.18it/s]


Applying on dataset 20


100%|███████████████████████████████████████████| 40/40 [00:12<00:00,  3.19it/s]
More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`). Consider using `matplotlib.pyplot.close()`.


Applying on dataset 30


100%|███████████████████████████████████████████| 40/40 [00:15<00:00,  2.56it/s]


Applying on dataset 40


100%|███████████████████████████████████████████| 40/40 [00:17<00:00,  2.33it/s]


Applying on dataset 50


100%|███████████████████████████████████████████| 40/40 [00:18<00:00,  2.15it/s]


In [10]:
# Save multiply imputed data as Python objects
with open(biopsy_path + "imputed.pickle", "wb") as handle:
    pickle.dump(biopsy_imputed, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [11]:
# Retrieve chain statistics from last iteration for the report
for p in props_name:
    print("p = {}".format(p / 100))
    tempmu = np.median(biopsy_imputed[p]["chainmean"][:3, :, -1], axis=1)
    print("Posterior median for mu = {}".format(tempmu))
    tempsig = np.median(biopsy_imputed[p]["chainstd"][:3, :, -1], axis=1)
    print("Posterior median for sigma = {}".format(tempsig))

p = 0.1
Posterior median for mu = [5.75609756 6.73333333 6.8       ]
Posterior median for sigma = [2.59290432 3.34182826 3.1607711 ]
p = 0.2
Posterior median for mu = [5.27840909 6.51694915 5.81034483]
Posterior median for sigma = [2.62672532 3.41037843 3.14369217]
p = 0.3
Posterior median for mu = [5.79666667 5.09313725 5.23170732]
Posterior median for sigma = [2.65738157 3.31962237 3.3730435 ]
p = 0.4
Posterior median for mu = [5.37317073 4.97887324 4.63709677]
Posterior median for sigma = [2.78678406 3.38362276 3.35905728]
p = 0.5
Posterior median for mu = [5.24163569 4.3681592  4.71621622]
Posterior median for sigma = [2.79340875 3.46671725 3.45770214]


## Apply complete case analysis

### `boston` dataset

In [10]:
# Prepare dictionary to store complete case data
boston_cc = {}

for p in tqdm(props_name):
    # Apply complete case analysis on each dataset
    boston_cc[p] = boston_incomp[p].dropna()
    print("Proportion = {}. Dimension of complete case data = {}".format(
        p/100, boston_cc[p].shape))

100%|████████████████████████████████████████████| 5/5 [00:00<00:00, 933.73it/s]

Proportion = 0.1. Dimension of complete case data = (467, 13)
Proportion = 0.2. Dimension of complete case data = (411, 13)
Proportion = 0.3. Dimension of complete case data = (365, 13)
Proportion = 0.4. Dimension of complete case data = (321, 13)
Proportion = 0.5. Dimension of complete case data = (262, 13)


In [11]:
# Create directory if not present yet
boston_cc_path = "../../data/toy-dataset/boston-complete-case/"
if not os.path.exists(boston_cc_path):
    os.mkdir(boston_cc_path)

# Store complete case data
for p in props_name:
    boston_cc[p].to_csv(boston_cc_path + "boston_{}.csv".format(p), 
                        index=False)

### `biopsy` dataset

In [12]:
# Prepare dictionary to store complete case data
biopsy_cc = {}

for p in tqdm(props_name):
    # Apply complete case analysis on each dataset
    biopsy_cc[p] = biopsy_incomp[p].dropna()
    print("Proportion = {}. Dimension of complete case data = {}".format(
        p/100, biopsy_cc[p].shape))

100%|███████████████████████████████████████████| 5/5 [00:00<00:00, 1159.61it/s]

Proportion = 0.1. Dimension of complete case data = (642, 9)
Proportion = 0.2. Dimension of complete case data = (583, 9)
Proportion = 0.3. Dimension of complete case data = (510, 9)
Proportion = 0.4. Dimension of complete case data = (432, 9)
Proportion = 0.5. Dimension of complete case data = (338, 9)


In [13]:
# Create directory if not present yet
biopsy_cc_path = "../../data/toy-dataset/biopsy-complete-case/"
if not os.path.exists(biopsy_cc_path):
    os.mkdir(biopsy_cc_path)

# Store complete case data
for p in props_name:
    biopsy_cc[p].to_csv(biopsy_cc_path + "biopsy_{}.csv".format(p), 
                        index=False)